In [4]:
import os
import pandas as pd
import numpy as np

from PIL import Image

data = pd.read_csv('all_.csv', index_col=0)

In [5]:
from ipywidgets import Textarea

def get_input(change):
    global Input
    Input = change["new"]

textarea = Textarea()
textarea.observe(get_input, names='value')
display(textarea)

Textarea(value='')

In [6]:
IN = iter(Input.split('\n')).__next__
def input():
    return IN()

N, M = map(int, input().split())

Graph = [[] for _ in range(4800)]

for i in range(M):
    a, b, direction = map(int, input().split())
    Graph[a].append((b, direction))

In [7]:
# 画像データがあるディレクトリを指定(localで動かす場合)
os.chdir("D:/worldmodeldata")

In [8]:
def node_int_to_string(node):
    node = str(node)
    temp = len(node)
    fill = 6 - temp
    fill_text = "0" * fill
    node = fill_text + node
    return node

def return_image_and_moveable_node(image, action):
    """
    Graph
    input image(node), action
    return image, next_node, and moveable_direction

    """
    current_node = int(image[:6])
    moveable_nodes = Graph[current_node]
    next_node = float('inf')
    
    
    print(f"今のnodeは{current_node}で動ける箇所は{Graph[current_node]}")
    
    for i in moveable_nodes:
        if i[1] == action:
            next_node = i[0]
    
    
    
    next_image = node_int_to_string(next_node) + "_" + str(action) + ".jpg"
    _next_image = Image.open(next_image)
    
    # ここでappendしてます。外に書いたほうがいいかもしれません。
    pictures.append(_next_image)
    next_moveable_nodes = Graph[next_node]
    
    return next_image, next_node, next_moveable_nodes

In [31]:
pictures = []
# next_image, next_node, next_moveable_nodes = return_image_and_moveable_node("001257_0.jpg", 1)

next_image = "000033_0.jpg"

for i in range(99):
    next_image, next_node, next_moveable_nodes = return_image_and_moveable_node(next_image, 0)

今のnodeは33で動ける箇所は[(34, 1), (32, 3), (223, 0)]
今のnodeは223で動ける箇所は[(224, 0), (33, 4)]
今のnodeは224で動ける箇所は[(225, 0), (223, 4)]
今のnodeは225で動ける箇所は[(226, 0), (224, 4)]
今のnodeは226で動ける箇所は[(227, 0), (225, 4)]
今のnodeは227で動ける箇所は[(228, 0), (226, 4)]
今のnodeは228で動ける箇所は[(229, 0), (227, 4)]
今のnodeは229で動ける箇所は[(230, 0), (228, 4)]
今のnodeは230で動ける箇所は[(231, 0), (229, 4)]
今のnodeは231で動ける箇所は[(232, 0), (230, 4)]
今のnodeは232で動ける箇所は[(233, 0), (231, 4)]
今のnodeは233で動ける箇所は[(234, 0), (232, 4)]
今のnodeは234で動ける箇所は[(235, 0), (233, 4)]
今のnodeは235で動ける箇所は[(236, 0), (234, 4)]
今のnodeは236で動ける箇所は[(237, 0), (235, 4)]
今のnodeは237で動ける箇所は[(238, 0), (236, 4)]
今のnodeは238で動ける箇所は[(239, 0), (237, 4)]
今のnodeは239で動ける箇所は[(240, 0), (238, 4)]
今のnodeは240で動ける箇所は[(241, 0), (239, 4)]
今のnodeは241で動ける箇所は[(242, 0), (240, 4)]
今のnodeは242で動ける箇所は[(243, 0), (241, 4)]
今のnodeは243で動ける箇所は[(244, 0), (242, 4)]
今のnodeは244で動ける箇所は[(245, 0), (243, 4)]
今のnodeは245で動ける箇所は[(322, 1), (1141, 3), (246, 0), (244, 4)]
今のnodeは246で動ける箇所は[(247, 0), (245, 4)]
今のnodeは247で動ける箇所は[(248,

In [32]:
pictures[0].save('anime.gif',save_all=True, append_images=pictures[1:], optimize=False, duration=500, loop=0)